In [2]:
pip install pygame


   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.6 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.6 MB 5.0 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/10.6 MB 4.6 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/10.6 MB 4.4 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.6 MB 4.3 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.6 MB 4.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.0/10.6 MB 4.1 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.6 MB 4.1 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.6 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.6 MB 4.1 MB/s eta 0:00:01
   ---------------------------------- ----- 9.2/10.6 MB 4.0 MB/s eta 0:00:01
   -------------------------------------- - 10.2/10.6 MB 4.0 MB/s eta 0:00:01
   -

In [3]:
import pygame
from pygame import mixer

# Initialize pygame and mixer
pygame.init()
mixer.init()

# Define the key-to-note mapping
KEY_TO_NOTE = {
    pygame.K_a: "C", pygame.K_s: "D", pygame.K_d: "E", pygame.K_f: "F",
    pygame.K_g: "G", pygame.K_h: "A", pygame.K_j: "B",
    pygame.K_k: "C_high",  # C one octave higher
}

# Load the sound files (ensure these .wav files are in the same directory or specify the path)
NOTES = {
    "C": "C.wav",
    "D": "D.wav",
    "E": "E.wav",
    "F": "F.wav",
    "G": "G.wav",
    "A": "A.wav",
    "B": "B.wav",
    "C_high": "C_high.wav",
}

# Load all sound files into a dictionary
sounds = {note: mixer.Sound(file) for note, file in NOTES.items()}

# Set up the screen
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("Laptop Piano")

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Play a sound when a key is pressed
        elif event.type == pygame.KEYDOWN:
            if event.key in KEY_TO_NOTE:
                note = KEY_TO_NOTE[event.key]
                sounds[note].play()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No file 'C.wav' found in working directory 'C:\Users\tutel\Anaconda Python'.

In [1]:
import pygame as pg
import numpy as np

pg.init()
pg.mixer.init()
screen = pg.display.set_mode((1280, 720))
font = pg.font.SysFont("Impact", 48)

def synth(frequency, duration=1.5, sampling_rate=44100):
    frames = int(duration*sampling_rate)
    arr = np.cos(2*np.pi*frequency*np.linspace(0,duration, frames))
    arr = arr + np.cos(4*np.pi*frequency*np.linspace(0,duration, frames))
    arr = arr - np.cos(6*np.pi*frequency*np.linspace(0,duration, frames))
##    arr = np.clip(arr*10, -1, 1) # squarish waves
##    arr = np.cumsum(np.clip(arr*10, -1, 1)) # triangularish waves pt1
##    arr = arr+np.sin(2*np.pi*frequency*np.linspace(0,duration, frames)) # triangularish waves pt1
    arr = arr/max(np.abs(arr)) # triangularish waves pt1
    sound = np.asarray([32767*arr,32767*arr]).T.astype(np.int16)
    sound = pg.sndarray.make_sound(sound.copy())
    
    return sound


keylist = '123456789qwertyuioasdfghjklzxcvbnm,.'
notes_file = open("noteslist.txt")
file_contents = notes_file.read()
notes_file.close()
noteslist = file_contents.splitlines()

keymod = '0-='
notes = {} # dict to store samples
freq = 16.3516 # start frequency
posx, posy = 25, 25 #start position


for i in range(len(noteslist)):
    mod = int(i/36)
    key = keylist[i-mod*36]+str(mod) 
    sample = synth(freq)
    color = np.array([np.sin(i/25+1.7)*130+125,np.sin(i/30-0.21)*215+40, np.sin(i/25+3.7)*130+125])
    color = np.clip(color, 0, 255)
    notes[key] = [sample, noteslist[i], freq, (posx, posy), 255*color/max(color)]
    notes[key][0].set_volume(0.33)
    notes[key][0].play()
    notes[key][0].fadeout(100)
    freq = freq * 2 ** (1/12)
    posx = posx + 140
    if posx > 1220:
        posx, posy = 25, posy+56
        
    screen.blit(font.render(notes[key][1], 0, notes[key][4]), notes[key][3])
    pg.display.update()
    

running = 1
mod = 1
pg.display.set_caption("FinFET Synth - Change range: 0 - = // Play with keys: "+keylist )

keypresses = []
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT or (event.type == pg.KEYDOWN and event.key == pg.K_ESCAPE):
            running = False
        if event.type == pg.KEYDOWN:
            key = str(event.unicode)
            if key in keymod:
                mod = keymod.index(str(event.unicode))
            elif key in keylist:
                key = key+str(mod)
                notes[key][0].play()
                keypresses.append([1, notes[key][1], pg.time.get_ticks()])
                screen.blit(font.render(notes[key][1], 0, (255,255,255)), notes[key][3])
        if event.type == pg.KEYUP and str(event.unicode) != '' and str(event.unicode) in keylist:
            key = str(event.unicode)+str(mod)
            notes[key][0].fadeout(100)
            keypresses.append([0, notes[key][1], pg.time.get_ticks()])
            screen.blit(font.render(notes[key][1], 0, notes[key][4]), notes[key][3])

    pg.display.update()

pg.display.set_caption("Exporting sound sequence")
if len(keypresses) > 1:
    for i in range(len(keypresses)-1):
        keypresses[-i-1][2] = keypresses[-i-1][2] - keypresses[-i-2][2]
    keypresses[0][2] = 0 # first at zero

    with open("test.txt", "w") as file:
        for i in range(len(keypresses)):
            file.write(str(keypresses[i])+'\n') # separate lines for readability
    file.close()
    
pg.mixer.quit()
pg.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
